In [ ]:
import pandas as pd
import numpy as np
import urllib
import matplotlib.pyplot as plt

In [ ]:
all_urls=!cat 'fall11_urls.csv'

In [ ]:
from random import shuffle

In [ ]:
shuffle(all_urls)
all_urls[:50]

In [ ]:
from PIL import Image

In [ ]:
from multiprocessing import Pool,Value

In [ ]:
count = Value('i', 0)

In [ ]:
def save_patch(url):
    global count
    try:
        imagename=url.split('/')[-1].split('.jpg')[0]
        with Image.open(urllib.request.urlopen(url,timeout=1)) as img:
            aux_im=np.array(img)
            min_shape=int(min(aux_im.shape[0],aux_im.shape[1])/2)
            center=(int(aux_im.shape[0]/2),int(aux_im.shape[1]/2))
            aux_patch=aux_im[center[0]-min_shape:center[0]+min_shape,center[1]-min_shape:center[1]+min_shape]
        if aux_patch.std()>20:
            with count.get_lock():
                Image.fromarray(aux_patch).resize((256,256)).save('imagenet/{:05d}.jpg'.format(count.value))
                count.value += 1
            return 1
        return 0
    except Exception as e: 
        #print(e)
        return 0

In [ ]:
pool = Pool(12)

In [ ]:
i=0
while count.value<100000:
    im_url=all_urls[i]
    count.value+=np.array(pool.map(save_patch,all_urls[i:i+1200])).sum()
    i+=1200
    print(i,count.value)